In [1]:
import numpy as np
import random as rd
import copy as cp

In [2]:
data = []
with open('sudoku1.txt', 'r') as file:
    for x in file:
        a = list(map(int, x.split()))
        data.append(a)
data = np.array(data)
print(data)

[[8 0 6 0 0 0 1 0 7]
 [0 0 0 6 0 2 0 0 0]
 [0 5 3 0 0 4 8 0 6]
 [7 0 4 8 0 0 6 3 0]
 [0 0 0 0 0 0 0 9 0]
 [1 0 0 5 0 0 4 0 0]
 [0 0 1 2 0 0 7 0 9]
 [2 0 0 0 9 6 0 0 0]
 [0 7 0 0 1 0 0 8 0]]


In [44]:
class Genetic:
    def __init__(self, data):
        self.data = data
        self.length = 81 - np.count_nonzero(data)
        self.val = [x for x in range(1, 10)]
        rank_prob = []
        for i in range(1, 101):
            a = (100 - i + 1)/5050
            rank_prob.append(a)
        self.rank_prob = tuple(rank_prob)
    def Create(self):
        individual = cp.deepcopy(data)
        for i in range(9):
            for j in range(9):
                if individual[i][j] == 0:
                    individual[i][j] = rd.choice(self.val)
                else:
                    continue
        return individual - self.data
    def Crossover(self, in1, in2):
        in1 = in1.reshape(81)
        in2 = in2.reshape(81)
        k = np.random.choice(80)
        new_in1 = np.concatenate([in1[:k], in2[k:]])
        new_in2 = np.concatenate([in2[:k], in1[k:]])
        return new_in1.reshape(9,9), new_in2.reshape(9, 9)
    def Mutation(self, in1, k = 5):
        in1 = in1.reshape(81)
        for i in range(k):
            a = np.random.choice(80)
            if in1[a] != 0:
                in1[a] = rd.choice(self.val)
        return in1.reshape(9, 9)
    def fitness(self, individual):
        a = self.data + individual
        fitness_val = 0
        for col in a:
            unique, counts = np.unique(col, return_counts=True)
            b = dict(zip(unique, counts))
            for i in b:
                fitness_val = fitness_val + b.get(i) - 1
        c = a.T
        for col in c:
            unique, counts = np.unique(col, return_counts=True)
            b = dict(zip(unique, counts))
            for i in b:
                fitness_val = fitness_val + b.get(i) - 1
        for i in range(3):
            for j in range(3):
                sq = [c[m][n] for m in range(i*3, i*3 + 3) for n in range(j*3, j*3 + 3)]
                sq = np.array(sq)
                unique, counts = np.unique(sq, return_counts=True)
                b = dict(zip(unique, counts))
                for k in b:
                    fitness_val = fitness_val + b.get(k) - 1
        return fitness_val
    def Elitist_selection(self, population):
        population = sorted(population, key=self.fitness)
        a = len(population)
        b = int(a /4) + 1
        return population[:b]
    def Rank_selection(self, population):
        population = sorted(population, key=self.fitness)
        new_pop = rd.choices(population, cum_weights=self.rank_prob, k=25)
        new_pop = sorted(new_pop, key=self.fitness)
        return new_pop
    def Tournament_selection(self, population):
        np.random.shuffle(population)
        new_pop = []
        for i in range(int(len(population)/4)):
            pop = population[i*4 : i*4 + 4]
            pop = sorted(pop, key=self.fitness)
            new_pop.append(pop[0])
        new_pop = sorted(new_pop, key=self.fitness)
        return new_pop
    def Roulette_wheel_selection(self, population):
        fit_list = []
        for i in population:
            fit_list.append(self.fitness(i))
        prob_list = []
        fit_sum = sum(fit_list)
        for i in fit_list:
            prob_list.append(i/fit_sum)
        prob_tup = tuple(prob_list)
        new_pop = rd.choices(population, cum_weights=prob_tup, k=25)
        new_pop = sorted(new_pop, key=self.fitness)
        return new_pop
    def Ge_al(self, pop_size = 100, max_gen = 400, select_type = 1, mutation_rate = 0.3):
        pop = [self.Create() for _ in range(pop_size)]
        for i in range(max_gen):
            #Selection:
            if select_type == 1:
                pop = self.Elitist_selection(pop)
            elif select_type == 2:
                pop = self.Rank_selection(pop)
            elif select_type == 3:
                pop = self.Tournament_selection(pop)
            else:
                pop = self.Roulette_wheel_selection(pop)
            #Nếu có phần tử thỏa mãn:
            if self.fitness(pop[0]) == 0:
                return pop[0] + self.data
            #Nếu không có phần tử thỏa mãn:
            new_pop = []
            #Crossover and Mutation
            while(len(new_pop) < pop_size):
                parent_1 = rd.choice(pop)
                parent_2 = rd.choice(pop)
                for j in self.Crossover(parent_1, parent_2):
                    a = rd.random()
                    if a > mutation_rate:
                        new_pop.append(j)
                    else:
                        j = self.Mutation(j)
                        new_pop.append(j)
            pop = cp.deepcopy(new_pop)
        pop = sorted(pop, key=self.fitness)
        if self.fitness(pop[0]) == 0:
            return pop[0] + self.data
        else:
            print('Not found')
            print(self.fitness(pop[0]))
            return pop[0] + self.data            

In [46]:
b = Genetic(data)
print(b.Ge_al())

Not found
17
[[8 2 6 9 5 3 1 2 7]
 [9 1 7 6 8 2 5 3 4]
 [4 5 3 1 7 4 8 9 6]
 [7 6 4 8 1 2 6 3 5]
 [5 9 8 4 3 7 2 9 1]
 [1 3 2 5 6 9 4 7 8]
 [6 3 1 2 4 8 7 5 9]
 [2 8 9 7 9 6 4 1 3]
 [4 7 5 3 1 5 6 8 8]]
